In [19]:
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier

In [20]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.lable_data_with_extra_features`
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df = query_job.to_dataframe()
df.shape
# df.to_csv('train_data.csv')

Client creating using default project: jm-ebg


(102571, 28)

In [21]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,timeOnSite_past,day_of_week_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,ProspectId_conv,visitStartTime_conv,date_conv,day_diff,label,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,711925274.1675325695,1675325695,20230202,0.682301,5,desktop,Opera Software,Windows,Windows 7,...,None,<NA>,None,<NA>,0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,busy,Not Contactable,Not Contactable,Others
1,01304386-f4b0-4601-ba81-097b966095fe,2061620715.1671277346,1675582360,20230205,35.725837,1,mobile,OPPO,Android,Android 12,...,None,<NA>,None,<NA>,0,01304386-f4b0-4601-ba81-097b966095fe,call disconnected by client,Outbound Phone Call Activity,Not Contactable,Call Disconnected By Client - No Communication
2,019e7323-c132-4cf3-b97e-048b84ed435a,222679061.1675254861,1675254861,20230201,0.462754,4,mobile,Realme,Android,Android 11,...,None,<NA>,None,<NA>,0,019e7323-c132-4cf3-b97e-048b84ed435a,busy,Outbound Phone Call Activity,Not Contactable,Others
3,01fbcabe-8ce6-4234-8ee6-e616fdf22136,1529146629.1673193933,1677211898,20230224,24.002012,6,mobile,Vivo,Android,Android 11,...,None,<NA>,None,<NA>,0,01fbcabe-8ce6-4234-8ee6-e616fdf22136,intrested,Outbound Phone Call Activity,Call Back,intrested
4,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,1396493678.1675044228,1678425248,20230310,4.844952,6,mobile,Vivo,Android,Android 12.0.0,...,None,<NA>,None,<NA>,0,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,call drop,Not Contactable,Not Contactable,Call Disconnected By Client - No Communication


In [22]:
df['ft_campaign_past'].value_counts()

(direct)                                                  56386
link                                                      11015
Leads_Performance_Demat_Tier_2                             5062
Leads_Performance_Demat_Tier_1                             4675
Leads_Performance_Max_Advisory_Tier_1                      3260
                                                          ...  
PT_Conversion_Tier1_above_24_Interest_1_above24 – Copy        1
Leads_Performance_Demat_Tier1                                 1
pennystocks                                                   1
24041                                                         1
investingmistakes                                             1
Name: ft_campaign_past, Length: 142, dtype: int64

In [23]:
int_cols = list(df.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [24]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
float_cols = list(df.select_dtypes(include=[float]).columns)

In [25]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [26]:
def analysis_count(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [27]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [28]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df['device_category_past'].loc[df['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df['mobile_brand_name_past'].loc[df['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df['operating_system_past'].loc[df['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df['os_version_past'].loc[df['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df['browser_past'].loc[df['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df['city_past'].loc[df['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df['region_past'].loc[df['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df['ft_campaign_past'].loc[df['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df['ft_medium_past'].loc[df['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df['ft_source_past'].loc[df['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df['utm_term_placement_past'].loc[df['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [29]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,timeOnSite_past,day_of_week_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,ProspectId_conv,visitStartTime_conv,date_conv,day_diff,label,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,711925274.1675325695,1675325695,20230202,0.682301,5,desktop,(Others),Windows,(Others),...,None,<NA>,None,<NA>,0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,busy,Not Contactable,Not Contactable,Others
1,01304386-f4b0-4601-ba81-097b966095fe,2061620715.1671277346,1675582360,20230205,35.725837,1,mobile,OPPO,Android,Android 12,...,None,<NA>,None,<NA>,0,01304386-f4b0-4601-ba81-097b966095fe,call disconnected by client,Outbound Phone Call Activity,Not Contactable,Call Disconnected By Client - No Communication
2,019e7323-c132-4cf3-b97e-048b84ed435a,222679061.1675254861,1675254861,20230201,0.462754,4,mobile,Realme,Android,Android 11,...,None,<NA>,None,<NA>,0,019e7323-c132-4cf3-b97e-048b84ed435a,busy,Outbound Phone Call Activity,Not Contactable,Others
3,01fbcabe-8ce6-4234-8ee6-e616fdf22136,1529146629.1673193933,1677211898,20230224,24.002012,6,mobile,Vivo,Android,Android 11,...,None,<NA>,None,<NA>,0,01fbcabe-8ce6-4234-8ee6-e616fdf22136,intrested,Outbound Phone Call Activity,Call Back,intrested
4,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,1396493678.1675044228,1678425248,20230310,4.844952,6,mobile,Vivo,Android,Android 12.0.0,...,None,<NA>,None,<NA>,0,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,call drop,Not Contactable,Not Contactable,Call Disconnected By Client - No Communication


In [30]:
### replace the string int and float value
for i in cat_cols:
    df[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df[i].fillna(-99999999.000, inplace = True) 

In [31]:
df=df.sort_values(by=['label'],ascending=False)

In [32]:
df

,ProspectId,user_pseudo_id,visitStartTime,event_date,timeOnSite_past,day_of_week_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,ProspectId_conv,visitStartTime_conv,date_conv,day_diff,label,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
51285,b811fc07-7b87-45b5-ba91-f00239c5ce93,220026711.1673950011,1673950010,20230117,36.263544,3,mobile,Vivo,Android,Android 12,...,b811fc07-7b87-45b5-ba91-f00239c5ce93,1674118835,20230119,2,1,b811fc07-7b87-45b5-ba91-f00239c5ce93,ringing,Outbound Phone Call Activity,Not Contactable,Application Submitted
88660,4a1faed8-da64-4d5d-8be7-e77624ee39b3,1187336851.1673949493,1674886241,20230128,120.348178,7,mobile,Vivo,Android,Android 13,...,4a1faed8-da64-4d5d-8be7-e77624ee39b3,1675138665,20230131,3,1,4a1faed8-da64-4d5d-8be7-e77624ee39b3,call disconnected by client,WhatsApp Message,Value_Not_Found,call disconnected by client
88655,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,1187336851.1673949493,1678454179,20230310,56.390069,6,mobile,Vivo,Android,Android 13.0.0,...,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,1678513085,20230311,1,1,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,ringing,Inbound Phone Call Activity,Value_Not_Found,Ringing
88656,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,1187336851.1673949493,1678603897,20230312,78.734232,1,mobile,Vivo,Android,Android 13.0.0,...,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,1678795420,20230314,2,1,d95e4f71-12dc-4100-8c41-91d3c8e0edd3,ringing,Inbound Phone Call Activity,Value_Not_Found,Ringing
88657,da1579ad-6adf-42ff-8ee1-3cf8cd8fe319,1187336851.1673949493,1676722975,20230218,129.589008,7,mobile,Vivo,Android,Android 13.0.0,...,da1579ad-6adf-42ff-8ee1-3cf8cd8fe319,1676980213,20230221,3,1,da1579ad-6adf-42ff-8ee1-3cf8cd8fe319,invlid no,Outbound Phone Call Activity,Not Contactable,Invalid number
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34845,70f4903e-bf21-4f28-a9e6-972749da3975,2090267170.1674972202,1674984232,20230129,35.176319,1,mobile,Vivo,Android,Android 12,...,Value_Not_Found,-99999,Value_Not_Found,-99999,0,70f4903e-bf21-4f28-a9e6-972749da3975,short hang up,Outbound Phone Call Activity,Not Contactable,Call Disconnected By Client - No Communication
34844,70b32d02-4b9a-423f-9c67-16002c95ad4a,1359423135.1676657578,1676657578,20230217,0.417658,6,mobile,Apple,iOS,(Others),...,Value_Not_Found,-99999,Value_Not_Found,-99999,0,70b32d02-4b9a-423f-9c67-16002c95ad4a,HANG UP,Outbound Phone Call Activity,Not Contactable,Others
34843,70005fad-1080-43dc-ac71-e73edfc7a463,1718908449.1677434003,1677434003,20230226,1.385165,1,mobile,Xiaomi,Android,(Others),...,Value_Not_Found,-99999,Value_Not_Found,-99999,0,70005fad-1080-43dc-ac71-e73edfc7a463,9739664317 ===== customer DID not picked up TH...,Not Contactable,Not Contactable,Ringing
34842,6f800098-3af0-4c20-bc32-4d25c879623b,1673438141.1676690308,1676690308,20230218,0.126650,7,mobile,Samsung,Android,Android 11.0.0,...,Value_Not_Found,-99999,Value_Not_Found,-99999,0,6f800098-3af0-4c20-bc32-4d25c879623b,busy,Not Contactable,Not Contactable,Ringing


In [33]:
drop_cols = ['user_pseudo_id', 'visitStartTime', 'event_date','user_psudo_id_conv', 'ProspectId_conv', 'visitStartTime_conv',
       'date_conv', 'day_diff','device_category_past','ProspectID_LeadCreation','mx_Last_Activity_Notes']
df = df.drop(drop_cols, axis=1)
df=df.drop_duplicates(subset=['ProspectId'], keep='first')
df = df.drop(['ProspectId'], axis=1)

In [34]:
df

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,label,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
51285,36.263544,3,Vivo,Android,Android 12,Chrome,Gujarat,Ahmedabad,(Others),(Others),(Others),(Others),1,Outbound Phone Call Activity,Not Contactable,Application Submitted
88660,120.348178,7,Vivo,Android,Android 13,Chrome,Gujarat,Ahmedabad,(Others),(Others),(Others),(Others),1,WhatsApp Message,Value_Not_Found,call disconnected by client
88655,56.390069,6,Vivo,Android,Android 13.0.0,Chrome,Gujarat,Ahmedabad,(Others),(Others),(Others),(Others),1,Inbound Phone Call Activity,Value_Not_Found,Ringing
88657,129.589008,7,Vivo,Android,Android 13.0.0,Chrome,Gujarat,Ahmedabad,(Others),(Others),(Others),(Others),1,Outbound Phone Call Activity,Not Contactable,Invalid number
14696,56.029278,4,OnePlus,Android,Android 12.0.0,Chrome,Maharashtra,Mumbai,(Others),(Others),(Others),(Others),1,Last Owner Activity Disposition,Not Interested,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34862,0.799676,3,Xiaomi,Android,Android 12.0.0,Chrome,West Bengal,Kolkata,Search_Demat_Brokers_Top_Core_Exact_Tier_1,cpc,google,(Others),0,Not Contactable,Not Contactable,Ringing
34857,0.000000,4,Realme,Android,Android 10.0.0,Chrome,West Bengal,Siliguri,Leads_Performance_Demat_Tier_1,cpc,google,(Others),0,Not Contactable,Not Contactable,Not reachable
34851,0.852434,1,Samsung,Android,Android 13,Chrome,West Bengal,Kolkata,Search_Trading_Stock_Trading_Top_Core_Phrase_T...,cpc,google,stock market,0,Not Contactable,Not Contactable,Call Disconnected By Client - No Communication
34841,0.338704,1,Vivo,Android,Android 12,Chrome,Delhi,Delhi,Search_Trading_Top_Core_Exact_Tier_1,cpc,google,trading in stocks,0,Not Interested,Not Contactable,Others


In [35]:
df['label'].value_counts()

0    19824
1      418
Name: label, dtype: Int64

In [36]:
df.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'label',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [37]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
cat_cols

['mobile_brand_name_past',
 'operating_system_past',
 'os_version_past',
 'browser_past',
 'region_past',
 'city_past',
 'ft_campaign_past',
 'ft_medium_past',
 'ft_source_past',
 'utm_term_placement_past',
 'ProspectActivityName_Max',
 'mx_Last_Activity_done_by_Agent',
 'mx_Sub_Dispostion']

In [38]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df[i] = le.fit_transform(df[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [39]:
X1= df.drop(['label'], axis=1)
y1 = df.label

In [40]:
y1=np.floor(pd.to_numeric(y1, errors='coerce')).astype('Int64')
import numpy as np
y1 = y1.fillna(0).astype(np.int64, errors='ignore')
X1 = X1.fillna(0).astype(np.int64, errors='ignore')

In [41]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
oversample = SMOTE()
X1, y1 = oversample.fit_resample(X1, y1)

In [42]:
X_train, X_test, y_train, y_test=train_test_split(X1, y1, test_size=.25, stratify=y1)

In [43]:
from sklearn.preprocessing import StandardScaler

ss_train = MinMaxScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = MinMaxScaler()
X_test = ss_test.fit_transform(X_test)

In [44]:
#Fitting Logistic Regression to the training set  
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0,max_iter=1000000)  
classifier.fit(X_train, y_train) 

LogisticRegression(max_iter=1000000, random_state=0)

In [45]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
logreg = LinearSVC(max_iter=1000000)
model = CalibratedClassifierCV(logreg) 
model.fit(X_train, y_train) 
predictions = model.predict(X_test)

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [196]:
clf1 = RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1200)
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1200)

In [174]:
X_train

array([[0.00265252, 0.44444444, 0.5       , ..., 0.63157895, 0.84615385,
        0.46938776],
       [0.0729443 , 0.77777778, 0.375     , ..., 0.78947368, 0.84615385,
        0.46938776],
       [0.        , 0.44444444, 0.875     , ..., 0.78947368, 0.23076923,
        0.04761905],
       ...,
       [0.00132626, 0.33333333, 0.625     , ..., 0.63157895, 0.84615385,
        0.14965986],
       [0.        , 0.55555556, 0.5       , ..., 0.63157895, 0.84615385,
        0.46938776],
       [0.        , 0.55555556, 0.625     , ..., 0.63157895, 0.84615385,
        0.46938776]])

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = classifier.predict(X_test)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y_test, y_pred)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("roc auc : ", roc_auc_score(y_test, y_pred))
print("recall : ", recall_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred))
print("f1 score : ",f1 )

print ("----------------------")


y_pred = model.predict(X_test)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y_test, y_pred)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("roc auc : ", roc_auc_score(y_test, y_pred))
print("recall : ", recall_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred))
print("f1 score : ",f1)

accuracy :  0.781678773204197
[[4269  687]
 [1477 3479]]
roc auc :  0.781678773204197
recall :  0.7019774011299436
precision :  0.8350936149783965
f1 score :  0.7627713220784915
----------------------
accuracy :  0.7799636803874092
[[4200  756]
 [1425 3531]]
roc auc :  0.7799636803874093
recall :  0.7124697336561744
precision :  0.8236529041287614
f1 score :  0.7640376501135995


In [ ]:
clf1


In [56]:
filename = 'model/RF_Dhawal.pkl'

pickle.dump(clf1, open(filename, 'wb'))

In [38]:
#Predicting the test set result  
y_pred= clf1.predict(X_test) 
y_pred1= clf1.predict_proba(X_test) 

In [39]:
y_pred=y_test

In [40]:
y_pred1=y_pred1[:,1]

In [41]:
y_pred=list(y_pred)

In [42]:
y_pred1
for i in range(y_pred1.size):
    if y_pred1[i]>0.6250:
        y_pred1[i]=1
    elif y_pred1[i]<0.2750:
        y_pred1[i]=0
    else:
        y_pred1[i]=-1

In [43]:
y_pred1

array([0., 0., 0., ..., 0., 1., 0.])

In [44]:
a=[]
b=[]
for i in range(y_pred1.size):
    if y_pred1[i]==1:
        a.append(y_pred1[i])
        b.append(y_pred[i])
    elif y_pred1[i]==0:
        a.append(y_pred1[i])
        b.append(y_pred[i])
        

In [45]:
len(a)

42645

In [46]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(a, b)

TN, FP, FN, TP = confusion_matrix(a, b).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  20673
False Positive(FP) =  396
True Negative(TN)  =  20890
False Negative(FN) =  686


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(a, b))
print(confusion_matrix(a, b))
print("roc auc : ", roc_auc_score(a, b))
print("recall : ", recall_score(a, b))
print("precision : ", precision_score(a, b))


----------------------
accuracy :  0.9746277406495486
[[20890   396]
 [  686 20673]]
roc auc :  0.9746393071836105
recall :  0.9678823914977293
precision :  0.9812046134130713


In [48]:
f1_score(a,b)

0.9744979730366738

# Production 

In [47]:
import pandas as pd
df3 = pd.read_csv('1_2may_testing_labeled.csv',dtype={"user_pseudo_id":str})
df3.shape

(12091, 22)

In [48]:
int_cols = list(df3.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [49]:
cat_cols = list(df3.select_dtypes(include=['O']).columns)
float_cols = list(df3.select_dtypes(include=[float]).columns)

In [50]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [51]:
def analysis_count(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [52]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [53]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df3['device_category_past'].loc[df3['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df3['mobile_brand_name_past'].loc[df3['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df3['operating_system_past'].loc[df3['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df3['os_version_past'].loc[df3['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df3['browser_past'].loc[df3['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df3['city_past'].loc[df3['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df3['region_past'].loc[df3['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df3['ft_campaign_past'].loc[df3['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df3['ft_medium_past'].loc[df3['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df3['ft_source_past'].loc[df3['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df3['utm_term_placement_past'].loc[df3['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [54]:
### replace the string int and float value
for i in cat_cols:
    df3[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df3[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df3[i].fillna(-99999999.000, inplace = True) 

In [55]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df3[i] = le.fit_transform(df3[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [56]:
df3=df3.sort_values(by=['label'],ascending=False)

In [57]:
drop_cols = ['user_pseudo_id', 'visitStartTime', 'event_date','device_category_past','mx_Last_Activity_Notes']
df3 = df3.drop(drop_cols, axis=1)
df3=df3.drop_duplicates(subset=['ProspectId'], keep='first')
df3 = df3.drop(['ProspectId'], axis=1)

In [58]:
df3['label'].value_counts()

0    1832
1      25
Name: label, dtype: int64

In [59]:
X = df3.drop(['label'], axis=1)
y = df3.label

In [60]:
X=X.loc[:,['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past',
        'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion']]

In [61]:
X.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion'],
      dtype='object')

In [62]:
df.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'label',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [63]:
y=np.floor(pd.to_numeric(y, errors='coerce')).astype('Int64')
import numpy as np
y = y.fillna(0).astype(np.int64, errors='ignore')
X = X.fillna(0).astype(np.int64, errors='ignore')

In [64]:
ss_train = MinMaxScaler()
X= ss_train.fit_transform(X)

In [52]:
filename = 'model/RF_Dhawal.pkl'
model = pickle.load(open(filename, 'rb'))

In [65]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = classifier.predict(X)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y, y_pred)
print("accuracy : ", accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))
print("roc auc : ", roc_auc_score(y, y_pred))
print("recall : ", recall_score(y, y_pred))
print("precision : ", precision_score(y, y_pred))
print("f1 score : ",f1 )

y_pred = model.predict(X)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y, y_pred)
print("accuracy : ", accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))
print("roc auc : ", roc_auc_score(y, y_pred))
print("recall : ", recall_score(y, y_pred))
print("precision : ", precision_score(y, y_pred))
print("f1 score : ",f1 )


accuracy :  0.8540656973613355
[[1579  253]
 [  18    7]]
roc auc :  0.5709497816593886
recall :  0.28
precision :  0.026923076923076925
f1 score :  0.04912280701754386
accuracy :  0.8459881529348411
[[1564  268]
 [  18    7]]
roc auc :  0.5668558951965067
recall :  0.28
precision :  0.025454545454545455
f1 score :  0.04666666666666667


In [90]:
y1=y1.astype('int64')
y=y.astype('int64')

In [74]:
df3.head()

,day_of_week_past,timeOnSite_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion,label
0,3,0.000000,3,0,3,2,4,68,0,0,0,0,9,3,46,0
1,2,1.474086,0,0,0,2,30,47,15,6,1,0,10,5,40,0
2,2,0.188153,0,0,0,2,30,47,15,6,1,0,10,5,40,0
3,2,1.897437,0,0,0,2,30,47,15,6,1,0,10,5,40,0
4,2,21.324201,0,0,0,2,30,47,15,6,1,0,10,5,40,0


In [75]:
df.head()

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,label,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.682301,5,0,4,0,5,46,3,48,1,6,71,0,12,8,62
1,35.725837,1,4,1,5,2,26,102,0,0,0,0,0,15,8,20
2,0.462754,4,6,1,3,2,42,32,39,1,6,142,0,15,8,62
3,24.002012,6,9,1,3,2,43,61,65,1,6,0,0,15,2,113
4,4.844952,6,9,1,6,2,26,102,0,0,0,0,0,12,8,20


In [79]:
X.head()

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.000000,3,3,0,3,2,4,68,0,0,0,0,9,3,46
1,1.474086,2,0,0,0,2,30,47,15,1,6,0,10,5,40
2,0.188153,2,0,0,0,2,30,47,15,1,6,0,10,5,40
3,1.897437,2,0,0,0,2,30,47,15,1,6,0,10,5,40
4,21.324201,2,0,0,0,2,30,47,15,1,6,0,10,5,40
